# Moving to Keras

Conx allows you to easily move back and forth between Keras.

Consider this conx network:

In [2]:
from conx import Network, Layer, SGD

net = Network("XOR2")
net.add(Layer("input1", 2))
net.add(Layer("input2", 2))
net.add(Layer("hidden1", 2, activation="sigmoid"))
net.add(Layer("hidden2", 2, activation="sigmoid"))
net.add(Layer("shared-hidden", 2, activation="sigmoid"))
net.add(Layer("output1", 2, activation="sigmoid"))
net.add(Layer("output2", 2, activation="sigmoid"))

net.connect("input1", "hidden1")
net.connect("input2", "hidden2")
net.connect("hidden1", "shared-hidden")
net.connect("hidden2", "shared-hidden")
net.connect("shared-hidden", "output1")
net.connect("shared-hidden", "output2")

ds = [
    ([[0, 0],[0, 0]], [[0, 0],[0, 0]]),
    ([[0, 0],[1, 1]], [[1, 1],[1, 1]]),
    ([[1, 1],[0, 0]], [[1, 1],[1, 1]]),
    ([[1, 1],[1, 1]], [[0, 0],[0, 0]])
]
net.dataset.load(ds)

conx, version 3.4.0
Using Theano backend.


You can easily generate a Keras script by calling `network.generate_keras_script()`:

In [3]:
code = net.generate_keras_script()

In [4]:
print(code)

## Autogenerated by conx

import keras

kfunc = {} # dictionary to keep track of k's by layer

kfunc['input2'] = keras.layers.Input((2,), *(), **{'name': 'input2'})
k = kfunc['input2']
for f in [keras.layers.Dense(2, **{'activation': 'sigmoid', 'name': 'hidden2'})]:
    k = f(k)
kfunc['hidden2'] = k
kfunc['input1'] = keras.layers.Input((2,), *(), **{'name': 'input1'})
k = kfunc['input1']
for f in [keras.layers.Dense(2, **{'activation': 'sigmoid', 'name': 'hidden1'})]:
    k = f(k)
kfunc['hidden1'] = k
k = keras.layers.Concatenate()([kfunc[layer] for layer in ['hidden1', 'hidden2']])
for f in [keras.layers.Dense(2, **{'activation': 'sigmoid', 'name': 'shared-hidden'})]:
    k = f(k)
kfunc['shared-hidden'] = k
k = kfunc['shared-hidden']
for f in [keras.layers.Dense(2, **{'activation': 'sigmoid', 'name': 'output2'})]:
    k = f(k)
kfunc['output2'] = k
k = kfunc['shared-hidden']
for f in [keras.layers.Dense(2, **{'activation': 'sigmoid', 'name': 'output1'})]:
    k = f(k)
kfunc['output1'] 

You can execute the code, completely independently from conx:

In [5]:
exec(code)

In [6]:
model.compile(loss="mse", optimizer="adam")

But still use the dataset compiled by conx:

In [7]:
model.fit(net.dataset._inputs, net.dataset._targets)

Epoch 1/1
4/4 [==============================] - 0s 193us/step - loss: 0.5294 - output1_loss: 0.2598 - output2_loss: 0.2696


In [8]:
model.predict(net.dataset._inputs)

[array([[ 0.43090475,  0.62888992],
        [ 0.42773837,  0.63394147],
        [ 0.43681258,  0.6076315 ],
        [ 0.43367642,  0.61252534]], dtype=float32),
 array([[ 0.46765664,  0.70279586],
        [ 0.46612409,  0.71049929],
        [ 0.47000363,  0.67751324],
        [ 0.46848112,  0.68529409]], dtype=float32)]

# Moving from Keras

In [1]:
batch_size = 128
num_classes = 10
epochs = 12
img_rows, img_cols = (28, 28)

In [2]:
import keras.backend as K
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

Using Theano backend.


## Keras Imperative Interface

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

## Keras Functional Interface

In [5]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Flatten, Dense, Dropout

In [6]:
inputs = k = Input(input_shape, name="input")
k = Conv2D(32, kernel_size=(3, 3), activation='relu')(k)
k = Conv2D(64, (3, 3), activation='relu')(k)
k = MaxPool2D(pool_size=(2, 2))(k)
k = Dropout(0.25)(k)
k = Flatten()(k)
k = Dense(128, activation='relu')(k)
k = Dropout(0.5)(k)
k = Dense(num_classes, activation='softmax')(k)
model = Model(inputs=inputs, outputs=k)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
__________

## Conx Interface

In [7]:
from conx import Layer, Network, Conv2DLayer, MaxPool2DLayer, FlattenLayer

In [8]:
network = Network("MNIST-CNN")
network.add(Layer("input", input_shape))
network.add(Conv2DLayer("conv1", 32, (3, 3), activation='relu'))
network.add(Conv2DLayer("conv2", 64, (3, 3), activation='relu'))
network.add(MaxPool2DLayer("maxpool", (2, 2), dropout=0.25))
network.add(FlattenLayer("flatten"))
network.add(Layer("hidden", 128, activation='relu', dropout=0.5))
network.add(Layer("output", num_classes, activation='softmax'))

network.connect("input", "conv1")
network.connect("conv1", "conv2")
network.connect("conv2", "maxpool")
network.connect("maxpool", "flatten")
network.connect("flatten", "hidden")
network.connect("hidden", "output")

# or, because this is sequetial, and layers added in order:
# network.connect()

network.compile(loss="categorical_crossentropy",
                optimizer="adadelta")
network.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 26, 26, 32)        320       
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 24, 64)        18496     
_________________________________________________________________
maxpool (MaxPooling2D)       (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
hidden (Dense)               (None, 128)               1179776   
__________